##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tipos de extensión

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/extension_type"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/extension_type.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/extension_type.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/extension_type.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Preparación

In [ ]:
!pip install -q tf_nightly
import tensorflow as tf
import numpy as np
from typing import Tuple, List, Mapping, Union, Optional
import tempfile

## Tipos de extensión

Los tipos definidos por el usuario pueden hacer que los proyectos sean más legibles, modulares y fáciles de mantener. Sin embargo, la mayoría de las API de TensorFlow son compatibles con muy pocos tipos Python definidos por el usuario. Esto se aplica tanto a las API de alto nivel (por ejemplo, [Keras](https://www.tensorflow.org/guide/keras/overview), [tf.function](https://www.tensorflow.org/guide/function), [`tf.SavedModel`](https://www.tensorflow.org/guide/saved_model)) como a las API de bajo nivel (es decir, `tf.while_loop` y `tf.concat`). Los **tipos de extensión ** de TensorFlow se pueden usar para crear tipos orientados a objetos definidos por el usuario que funcionan sin problemas con las API de TensorFlow. Para crear un tipo de extensión, simplemente se debe definir una clase Python con `tf.experimental.ExtensionType` como base, y usar [anotaciones de tipo](https://www.python.org/dev/peps/pep-0484/) para especificar el tipo de cada campo.

In [ ]:
class TensorGraph(tf.experimental.ExtensionType):
  """A collection of labeled nodes connected by weighted edges."""
  edge_weights: tf.Tensor               # shape=[num_nodes, num_nodes]
  node_labels: Mapping[str, tf.Tensor]  # shape=[num_nodes]; dtype=any

class MaskedTensor(tf.experimental.ExtensionType):
  """A tensor paired with a boolean mask, indicating which values are valid."""
  values: tf.Tensor
  mask: tf.Tensor       # shape=values.shape; false for missing/invalid values.

class CSRSparseMatrix(tf.experimental.ExtensionType):
  """Compressed sparse row matrix (https://en.wikipedia.org/wiki/Sparse_matrix)."""
  values: tf.Tensor     # shape=[num_nonzero]; dtype=any
  col_index: tf.Tensor  # shape=[num_nonzero]; dtype=int64
  row_index: tf.Tensor  # shape=[num_rows+1]; dtype=int64

La clase `tf.experimental.ExtensionType` de base funciona de forma similar a [`typing.NamedTuple`](https://docs.python.org/3/library/typing.html#typing.NamedTuple) y [`@dataclasses.dataclass`](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass) dela biblioteca estándar de Python. En particular, agrega automáticamente un constructor y métodos especiales (como `__repr__` y `__eq__`) en función de las anotaciones de tipo de campo.

Por lo general, los tipos de extensión pertenecen a una de estas dos categorías:

- ***Estructuras de datos***, que agrupan una colección de valores relacionados y pueden ofrecer operaciones útiles con base en esos valores. Las estructuras de datos pueden ser bastante generales (como en el ejemplo de `TensorGraph` que se muestra más arriba); o estar muy personalizadas para un modelo específico.

- ***Tipos similares a tensores***, que especializan o amplían el concepto de "Tensor". Los tipos de esta categoría tienen `rank`, `shape` y, generalmente, `dtype`; y lo más lógico es usarlos con operaciones de Tensor `tf.stack`, `tf.add` o `tf.matmul`). `MaskedTensor` y `CSRSparseMatrix` son ejemplos de tipos similares a tensores.

## API compatibles

Los tipos de extensión son compatibles con las siguientes API de TensorFlow:

- **Keras**: los tipos de extensión se pueden usar como entradas o salidas para `Models` y `Layers` de Keras.
- **`tf.data.Dataset`**: los tipos de extensión se pueden incluir en `Datasets`, y devolver por `Iterators` de conjunto de datos.
- **TensorFlow Hub**: los tipos de extensión se pueden usar como entradas o salidas para módulos de `tf.hub`.
- **SavedModel**: Los tipos de extensión se pueden usar como entradas o salidas para funciones `SavedModel`.
- **`tf.function`**: los tipos de extensión se pueden usar como argumentos y devuelven valores para funciones envueltas con el decorador `@tf.function`.
- **Bucles while**: los tipos de extensión se pueden usar como variables de bucle en `tf.while_loop`, y se pueden usar como argumentos y devolver valores para el cuerpo del bucle while.
- **Condicionales**: los tipos de extensión se pueden seleccionar de forma condicional mediante el uso de `tf.cond` y `tf.case`.
- **`tf.py_function`**: los tipos de extensión se pueden usar como argumentos y devolver valores para el argumento `func` a `tf.py_function`.
- **Operaciones de Tensor**: Extension types can be extended to support most TensorFlow ops that accept Tensor inputs (such as `tf.matmul`, `tf.gather`, and `tf.reduce_sum`). Go to the "*Dispatch*" section below for more information.
- **Estrategia de distribución**: los tipos de extensión se pueden usar como valores por réplica.

Para obtener más información, consulte la sección "API de TensorFlow compatibles con ExtensionTypes" a continuación.


## Requisitos


### Tipos de campo

Se deben declarar todos los campos, también conocidos como variables de instancia, y se debe proporcionar una anotación de tipo por cada campo. Se admiten las siguientes anotaciones de tipo:

Tipo | Ejemplo
--- | ---
Enteros de Python | `i: int`
Flotantes de Python | `f: float`
Cadenas de Python | `s: str`
Booleanos de Python | `b: bool`
`None` de Python | `n: None`
[Formas de tensor](https://www.tensorflow.org/api_docs/python/tf/TensorShape) | `shape: tf.TensorShape`
[`dtype` de tensor](https://www.tensorflow.org/api_docs/python/tf/dtypes/DType) | `dtype: tf.DType`
[Tensores](https://www.tensorflow.org/api_docs/python/tf/Tensor) | `t: tf.Tensor`
[Tipos de extensión](https://www.tensorflow.org/api_docs/python/tf/experimental/ExtensionType) | `mt: MyMaskedTensor`
[Tensores irregulares](https://www.tensorflow.org/api_docs/python/tf/RaggedTensor) | `rt: tf.RaggedTensor`
[Tensores dispersos](https://www.tensorflow.org/api_docs/python/tf/sparse/SparseTensor) | `st: tf.SparseTensor`
[Segmentos indexados](https://www.tensorflow.org/api_docs/python/tf/IndexedSlices) | `s: tf.IndexedSlices`
[Tensores opcionales](https://www.tensorflow.org/api_docs/python/tf/experimental/Optional) | `o: tf.experimental.Optional`
[Uniones de tipo](https://docs.python.org/3/library/typing.html#typing.Union) | `int_or_float: typing.Union[int, float]`
[Tuplas](https://docs.python.org/3/library/typing.html#typing.Tuple) | `params: typing.Tuple[int, float, tf.Tensor, int]`
[Tuplas de longitud variable](https://docs.python.org/3/library/typing.html#typing.Tuple) | `lengths: typing.Tuple[int, ...]`
[Asignaciones](https://docs.python.org/3/library/typing.html#typing.Mapping) | `tags: typing.Mapping[str, tf.Tensor]`
[Valores opcionales](https://docs.python.org/3/library/typing.html#typing.Optional) | `weight: typing.Optional[tf.Tensor]`

### Mutabilidad

Los tipos de extensión deben ser inmutables. Esto garantiza que los mecanismos de trazado de gráficos de TensorFlow puedan hacer un seguimiento adecuado. Si de pronto necesita mutar un valor de tipo de extensión, analice la posibilidad de definir métodos que transformen valores. Por ejemplo, en lugar de definir un método `set_mask` para mutar un `MaskedTensor`, puede definir un método `replace_mask` que devuelva un `MaskedTensor` nuevo:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def replace_mask(self, new_mask):
      self.values.shape.assert_is_compatible_with(new_mask.shape)
      return MaskedTensor(self.values, new_mask)

## Funcionalidad agregada mediante `ExtensionType`

La clase `ExtensionType` de base ofrece la siguiente funcionalidad:

- Un constructor (`__init__`).
- Un método de representación imprimible (`__repr__`).
- Operadores de igualdad y desigualdad (`__eq__`).
- Un método de validación (`__validate__`).
- Inmutabilidad forzada.
- Un `TypeSpec` anidado.
- Compatibilidad con envío de API de Tensor.

Consulte la sección "Personalización de `ExtensionType`" a continuación para obtener más información sobre cómo personalizar esta funcionalidad.

### Constructor

El constructor agregado por `ExtensionType` toma cada campo como un argumento con nombre (en el orden en que aparecen en la definición de la clase). Este constructor verificará el tipo de cada parámetro y los convertirá cuando sea necesario. En particular, los campos `Tensor` se convierten usando `tf.convert_to_tensor`; los campos `Tuple` se convierten en `tuple`; y los campos `Mapping` se convierten en diccionarios inmutables.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

# Constructor takes one parameter for each field.
mt = MaskedTensor(values=[[1, 2, 3], [4, 5, 6]],
                  mask=[[True, True, False], [True, False, True]])

# Fields are type-checked and converted to the declared types.
# For example, `mt.values` is converted to a Tensor.
print(mt.values)

Si un valor de campo no se puede convertir al tipo declarado, el constructor genera un `TypeError`:

In [ ]:
try:
  MaskedTensor([1, 2, 3], None)
except TypeError as e:
  print(f"Got expected TypeError: {e}")

El valor predeterminado de un campo se puede especificar estableciendo su valor a nivel de clase:

In [ ]:
class Pencil(tf.experimental.ExtensionType):
  color: str = "black"
  has_erasor: bool = True
  length: tf.Tensor = 1.0

Pencil()

In [ ]:
Pencil(length=0.5, color="blue")

### Representación imprimible

`ExtensionType` agrega un método de representación imprimible predeterminado (`__repr__`) que incluye el nombre de la clase y el valor de cada campo:


In [ ]:
print(MaskedTensor(values=[1, 2, 3], mask=[True, True, False]))

### Operadores de igualdad

`ExtensionType` agrega los operadores de igualdad predeterminados (`__eq__` y `__ne__`) que consideran que dos valores son iguales si tienen el mismo tipo y todos sus campos son iguales. Los campos Tensor se consideran iguales si tienen la misma forma y son iguales elementalmente para todos los elementos.

In [ ]:
a = MaskedTensor([1, 2], [True, False])
b = MaskedTensor([[3, 4], [5, 6]], [[False, True], [True, True]])
print(f"a == a: {a==a}")
print(f"a == b: {a==b}")
print(f"a == a.values: {a==a.values}")

**Nota:** Si algún campo contiene un `Tensor`, entonces `__eq__` puede devolver un `Tensor` booleano escalar (en vez de un valor booleano de Python).

### Método de validación

`ExtensionType` agrega un método `__validate__`, que se puede anular para ejecutar verificaciones de validación en los campos. Se ejecuta después de llamar al constructor y verificar el tipo de los campos y convertirlos a los tipos declarados, por lo que puede asumir que todos los campos tienen sus tipos declarados.

El siguiente ejemplo actualiza `MaskedTensor` para validar los valores `shape` y `dtype` de sus campos:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  """A tensor paired with a boolean mask, indicating which values are valid."""
  values: tf.Tensor
  mask: tf.Tensor
  def __validate__(self):
    self.values.shape.assert_is_compatible_with(self.mask.shape)
    assert self.mask.dtype.is_bool, 'mask.dtype must be bool'

In [ ]:
try:
  MaskedTensor([1, 2, 3], [0, 1, 0])  # Wrong `dtype` for mask.
except AssertionError as e:
  print(f"Got expected AssertionError: {e}")

In [ ]:
try:
  MaskedTensor([1, 2, 3], [True, False])  # shapes don't match.
except ValueError as e:
  print(f"Got expected ValueError: {e}")

### Inmutabilidad forzada

`ExtensionType` anula los métodos `__setattr__` y `__delattr__` para impedir la mutación, lo que garantiza que los valores de tipo de extensión sean inmutables.

In [ ]:
mt = MaskedTensor([1, 2, 3], [True, False, True])

In [ ]:
try:
  mt.mask = [True, True, True]
except AttributeError as e:
  print(f"Got expected AttributeError: {e}")

In [ ]:
try:
  mt.mask[0] = False
except TypeError as e:
  print(f"Got expected TypeError: {e}")

In [ ]:
try:
  del mt.mask
except AttributeError as e:
  print(f"Got expected AttributeError: {e}")

### TypeSpec anidado

Cada clase `ExtensionType` tiene una clase `TypeSpec` correspondiente, que se crea automáticamente y se almacena como `<extension_type_name>.Spec`.

Esta clase recoge toda la información de un valor *excepto* por los valores de cualquier tensor anidado. En particular, la clase `TypeSpec` de un valor se crear reemplazando cualquier Tensor, ExtensionType o CompositeTensor anidado por su `TypeSpec`.


In [ ]:
class Player(tf.experimental.ExtensionType):
  name: tf.Tensor
  attributes: Mapping[str, tf.Tensor]

anne = Player("Anne", {"height": 8.3, "speed": 28.1})
anne_spec = tf.type_spec_from_value(anne)
print(anne_spec.name)  # Records `dtype` and `shape`, but not the string value.
print(anne_spec.attributes)  # Records keys and TensorSpecs for values.

Los valores `TypeSpec` se pueden construir de forma explícita o se pueden generar a partir de un valor `ExtensionType` mediante el uso de `tf.type_spec_from_value`:

In [ ]:
spec1 = Player.Spec(name=tf.TensorSpec([], tf.float32), attributes={})
spec2 = tf.type_spec_from_value(anne)

TensorFlow usa los valores `TypeSpec` para dividir valores en un **componente estático** y un **componente dinámico**:

- El **componente estático** (que se fija al momento de construir el gráfico) está codificado con un `tf.TypeSpec`.
- El **componente dinámico** (que puede variar cada vez que se ejecuta el gráfico) está codificado como una lista de `tf.Tensor`.

Por ejemplo, `tf.function` vuelve sobre su función envuelta cada vez que un argumento tiene un `TypeSpec` que no ha visto antes:

In [ ]:
@tf.function
def anonymize_player(player):
  print("<<TRACING>>")
  return Player("<anonymous>", player.attributes)

In [ ]:
# Function gets traced (first time the function has been called):
anonymize_player(Player("Anne", {"height": 8.3, "speed": 28.1}))

In [ ]:
# Function does NOT get traced (same TypeSpec: just tensor values changed)
anonymize_player(Player("Bart", {"height": 8.1, "speed": 25.3}))

In [ ]:
# Function gets traced (new TypeSpec: keys for attributes changed):
anonymize_player(Player("Chuck", {"height": 11.0, "jump": 5.3}))

Para obtener más información, consulte la [Guía de tf.function](https://www.tensorflow.org/guide/function#rules_of_tracing).

## Personalización de `ExtensionType`

Además de declarar los campos y sus tipos, los tipos de extensión pueden hacer lo siguiente:

- Anular la representación imprimible predeterminada (`__repr__`).
- Definir los métodos.
- Definir `classmethod` y `staticmethod`.
- Definir las propiedades.
- Anular el constructor predeterminado (`__init__`).
- Anular el operador de igualdad predeterminado (`__eq__`).
- Definir los operadores (como `__add__` y `__lt__`).
- Declarar los valores predeterminados de los campos.
- Definir las subclases.


### Anular la representación imprimible predeterminada

Puede anular este operador de conversión de cadena predeterminado para tipos de extensión. El siguiente ejemplo actualiza la clase `MaskedTensor` para generar una representación de cadena más legible cuando los valores se imprimen en modo Eager.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  """A tensor paired with a boolean mask, indicating which values are valid."""
  values: tf.Tensor
  mask: tf.Tensor       # shape=values.shape; false for invalid values.

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

def masked_tensor_str(values, mask):
  if isinstance(values, tf.Tensor):
    if hasattr(values, 'numpy') and hasattr(mask, 'numpy'):
      return f'<MaskedTensor {masked_tensor_str(values.numpy(), mask.numpy())}>'
    else:
      return f'MaskedTensor(values={values}, mask={mask})'
  if len(values.shape) == 1:
    items = [repr(v) if m else '_' for (v, m) in zip(values, mask)]
  else:
    items = [masked_tensor_str(v, m) for (v, m) in zip(values, mask)]
  return '[%s]' % ', '.join(items)

mt = MaskedTensor(values=[[1, 2, 3], [4, 5, 6]],
                  mask=[[True, True, False], [True, False, True]])
print(mt)

### Definir los métodos

Los tipos de extensión pueden definir los métodos, del mismo modo que cualquier clase normal de Python. Por ejemplo, el tipo `MaskedTensor` podría definir un método `with_default` que devuelva una copia de `self` con valores enmascarados reemplazados por un valor `default` determinado. De forma opcional, los métodos se pueden anotar con el decorador `@tf.function`.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def with_default(self, default):
    return tf.where(self.mask, self.values, default)

MaskedTensor([1, 2, 3], [True, False, True]).with_default(0)

### Definir `classmethod` y `staticmethod`

Los tipos de extensión pueden usar los decoradores `@classmethod` y `@staticmethod` para definir los métodos. Por ejemplo, el tipo `MaskedTensor` podría definir un método de fábrica que enmascare cualquier elemento con un valor determinado:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  @staticmethod
  def from_tensor_and_value_to_mask(values, value_to_mask):
    return MaskedTensor(values, values != value_to_mask)

x = tf.constant([[1, 0, 2], [3, 0, 0]])
MaskedTensor.from_tensor_and_value_to_mask(x, 0)

### Definir las propiedades

Los tipos de extensión pueden usar el decorador `@property` para definir las propiedades, como cualquier clase normal de Python. Por ejemplo, el tipo `MaskedTensor` podría definir una propiedad `dtype` que es una abreviatura para el `dtype` de los valores:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  @property
  def dtype(self):
    return self.values.dtype

MaskedTensor([1, 2, 3], [True, False, True]).dtype

### Anular el constructor predeterminado

Puede anular el constructor predeterminado de los tipos de extensión. Los constructores personalizados deben establecer un valor para cada campo declarado; y después de que se devuelva el constructor personalizado, se verificará el tipo de todos los campos y se convertirán los valores como se describe anteriormente.

In [ ]:
class Toy(tf.experimental.ExtensionType):
  name: str
  price: tf.Tensor
  def __init__(self, name, price, discount=0):
    self.name = name
    self.price = price * (1 - discount)

print(Toy("ball", 5.0, discount=0.2))  # On sale -- 20% off!

Otra opción es dejar el constructor predeterminado como está, pero agregar uno o más métodos de fábrica. Por ejemplo:

In [ ]:
class Toy(tf.experimental.ExtensionType):
  name: str
  price: tf.Tensor

  @staticmethod
  def new_toy_with_discount(name, price, discount):
    return Toy(name, price * (1 - discount))

print(Toy.new_toy_with_discount("ball", 5.0, discount=0.2))

### Anular el operador de igualdad predeterminado (`__eq__`).

Puede anular el operador `__eq__` predeterminado de los tipos de extensión. En el siguiente ejemplo, se actualiza `MaskedTensor` para que ignore los elementos enmascarados cuando los compare para comprobar la igualdad.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  def __eq__(self, other):
    result = tf.math.equal(self.values, other.values)
    result = result | ~(self.mask & other.mask)
    return tf.reduce_all(result)

x = MaskedTensor([1, 2, 3, 4], [True, True, False, True])
y = MaskedTensor([5, 2, 0, 4], [False, True, False, True])
print(x == y)

**Nota:** Por lo general, no es ncesario anular `__ne__`, ya que la implementación predeterminada simplemente llama `__eq__` y niega el resultado.

### Usar referencias directas

Si aún no se ha definido el tipo de un campo, puede usar en su lugar una cadena que contenga el nombre del tipo. En el siguiente ejemplo, la cadena `"Node"` se usa para anotar el campo `children` porque el tipo `Node` aún no se ha definido (completamente).


In [ ]:
class Node(tf.experimental.ExtensionType):
  value: tf.Tensor
  children: Tuple["Node", ...] = ()

Node(3, [Node(5), Node(2)])

### Definir las subclases

Los tipos de extensión se pueden subclasificar con la sintaxis estándar de Python. Las subclases de tipo de extensión pueden agregar campos, métodos y propiedades nuevos; y pueden anular el constructor, la representación imprimible y el operador de igualdad. En el siguiente ejemplo, se define una clase `TensorGraph` básica que usa tres campos `Tensor` para codificar un conjunto de perímetros entre nodos. Luego, define una subclase que agrega un campo `Tensor` para registrar un "valor de característica" para cada nodo. La subclase también define un método para propagar los valores de las características por los perímetros.

In [ ]:
class TensorGraph(tf.experimental.ExtensionType):
  num_nodes: tf.Tensor
  edge_src: tf.Tensor   # edge_src[e] = index of src node for edge e.
  edge_dst: tf.Tensor   # edge_dst[e] = index of dst node for edge e.

class TensorGraphWithNodeFeature(TensorGraph):
  node_features: tf.Tensor  # node_features[n] = feature value for node n.

  def propagate_features(self, weight=1.0) -> 'TensorGraphWithNodeFeature':
    updates = tf.gather(self.node_features, self.edge_src) * weight
    new_node_features = tf.tensor_scatter_nd_add(
        self.node_features, tf.expand_dims(self.edge_dst, 1), updates)
    return TensorGraphWithNodeFeature(
        self.num_nodes, self.edge_src, self.edge_dst, new_node_features)

g = TensorGraphWithNodeFeature(  # Edges: 0->1, 4->3, 2->2, 2->1
    num_nodes=5, edge_src=[0, 4, 2, 2], edge_dst=[1, 3, 2, 1],
    node_features=[10.0, 0.0, 2.0, 5.0, -1.0, 0.0])

print("Original features:", g.node_features)
print("After propagating:", g.propagate_features().node_features)

### Definir los campos privados

Los campos de un tipo de extensión pueden marcarse como privados si se les antepone un guion bajo (de acuerdo con las convenciones estándar de Python). Esto no afecta en lo más mínimo la forma en que TensorFlow trata los campos; simplemente sirve para indicarles a los usuarios del tipo de extensión que se trata de campos privados.


### Personalizar `TypeSpec` de `ExtensionType`

Cada clase `ExtensionType` tiene una clase `TypeSpec` correspondiente, que se crea automáticamente y se almacena como `<extension_type_name>.Spec`. Para obtener más información, consulte la sección "TypeSpec anidado" que se menciona anteriormente.

Para personalizar `TypeSpec`, sencillamente debe definir su propia clase anidada con el nombre de `Spec`, y `ExtensionType` la usará como base para la clase `TypeSpec` que se construye automáticamente. Puede personalizar la clase `Spec` de la siguiente manera:

- Anulando la representación imprimible predeterminada.
- Anulando el constructor predeterminado.
- Definiendo los métodos, `classmethod`, `staticmethod`, y las propiedades.

En el siguiente ejemplo, se personaliza la clase `MaskedTensor.Spec` para simplificar su uso:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  shape = property(lambda self: self.values.shape)
  dtype = property(lambda self: self.values.dtype)

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  def with_values(self, new_values):
    return MaskedTensor(new_values, self.mask)

  class Spec:
    def __init__(self, shape, dtype=tf.float32):
      self.values = tf.TensorSpec(shape, dtype)
      self.mask = tf.TensorSpec(shape, tf.bool)

    def __repr__(self):
      return f"MaskedTensor.Spec(shape={self.shape}, dtype={self.dtype})"

    shape = property(lambda self: self.values.shape)
    dtype = property(lambda self: self.values.dtype)

**Nota**: La clase `Spec` personalizada no puede usar ninguna variable de instancia que no haya sido declarada en el `ExtensionType` original.

## Envío de API de Tensor

Los tipos de extensión pueden ser "como tensores", en el sentido de que especializan o amplían la interfaz definida por el tipo `tf.Tensor`. Entre los ejemplos de tipos de extensión similares a los tensores se incluyen `RaggedTensor`, `SparseTensor` y `MaskedTensor`. Los ***decoradores de envío*** se pueden usar para anular el comportamiento predeterminado de las operaciones de TensorFlow cuando se aplican a tipos de extensión similares a los tensores. Actualmente, TensorFlow define tres decoradores de envío:

- `@tf.experimental.dispatch_for_api(tf_api)`
- `@tf.experimental.dispatch_for_unary_elementwise_apis(x_type)`
- `@tf.experimental.dispatch_for_binary_elementwise_apis(x_type, y_type)`

### Envío para una sola API

El decorador `tf.experimental.dispatch_for_api` anula el comportamiento predeterminado de una operación especificada de TensorFlow cuando se llama con la firma especificada. Por ejemplo, puede usar este decorador para especificar cómo debe procesar `tf.stack` los valores de `MaskedTensor`:

In [ ]:
@tf.experimental.dispatch_for_api(tf.stack)
def masked_stack(values: List[MaskedTensor], axis = 0):
  return MaskedTensor(tf.stack([v.values for v in values], axis),
                      tf.stack([v.mask for v in values], axis))

Esto anula la implementación predeterminada de `tf.stack` siempre que se llame con una lista de valores de `MaskedTensor` (ya que el argumento `values` se anota con `typing.List[MaskedTensor]`):

In [ ]:
x = MaskedTensor([1, 2, 3], [True, True, False])
y = MaskedTensor([4, 5, 6], [False, True, True])
tf.stack([x, y])

Para permitir que `tf.stack` maneje listas de valores `MaskedTensor` y `Tensor` combinados, puede refinar la anotación de tipo para el parámetro `values` y actualizar el cuerpo de la función debidamente:

In [ ]:
tf.experimental.unregister_dispatch_for(masked_stack)

def convert_to_masked_tensor(x):
  if isinstance(x, MaskedTensor):
    return x
  else:
    return MaskedTensor(x, tf.ones_like(x, tf.bool))

@tf.experimental.dispatch_for_api(tf.stack)
def masked_stack_v2(values: List[Union[MaskedTensor, tf.Tensor]], axis = 0):
  values = [convert_to_masked_tensor(v) for v in values]
  return MaskedTensor(tf.stack([v.values for v in values], axis),
                      tf.stack([v.mask for v in values], axis))
x = MaskedTensor([1, 2, 3], [True, True, False])
y = tf.constant([4, 5, 6])
tf.stack([x, y, x])

Para acceder a una lista de las API que se pueden anular, consulte la documentación de API de `tf.experimental.dispatch_for_api`.

### Envío para todas las API elementales unarias

El decorador `tf.experimental.dispatch_for_unary_elementwise_apis` anula el comportamiento predeterminado de ***todas*** las operaciones elementales unarias (como `tf.math.cos`) siempre que el valor del primer argumento (generalmente nombrado como `x`) coincida con la anotación de tipo `x_type`. La función decorada debe tomar dos argumentos:

- `api_func`: una función que toma un solo parámetro y ejecuta una operación elemental (por ejemplo, `tf.abs`).
- `x`: el primer argumento de la operación elemental.

El siguiente ejemplo actualiza todas las operaciones elementales unarias para manejar el tipo `MaskedTensor`:

In [ ]:
 @tf.experimental.dispatch_for_unary_elementwise_apis(MaskedTensor)
 def masked_tensor_unary_elementwise_api_handler(api_func, x):
   return MaskedTensor(api_func(x.values), x.mask)

Ahora, esta función se usará siempre que se llame una operación elemental unaria con `MaskedTensor`.

In [ ]:
 x = MaskedTensor([1, -2, -3], [True, False, True])
 print(tf.abs(x))

In [ ]:
print(tf.ones_like(x, dtype=tf.float32))

### Envío para todas las API elementales binarias

De forma similar, se puede usar `tf.experimental.dispatch_for_binary_elementwise_apis` para actualizar todas las operaciones elementales binarias para manejar el tipo `MaskedTensor`:


In [ ]:
@tf.experimental.dispatch_for_binary_elementwise_apis(MaskedTensor, MaskedTensor)
def masked_tensor_binary_elementwise_api_handler(api_func, x, y):
  return MaskedTensor(api_func(x.values, y.values), x.mask & y.mask)

In [ ]:
x = MaskedTensor([1, -2, -3], [True, False, True])
y = MaskedTensor([[4], [5]], [[True], [False]])
tf.math.add(x, y)

Para obtener una lista de las API elementales anuladas, consulte la documentación de API de `tf.experimental.dispatch_for_unary_elementwise_apis` y `tf.experimental.dispatch_for_binary_elementwise_apis`.

## `ExtensionType` por lotes

Un `ExtensionType` se puede *procesar por lotes* si una única instancia se puede usar para representar un lote de valores. Por lo general, esto se consigue al agregar dimensiones de lote a todos los `Tensor` anidados. Las siguientes API de TensorFlow exigen que cualquier entrada de tipo de extensión se pueda procesar por lotes:

- `tf.data.Dataset` (`batch`, `unbatch`, `from_tensor_slices`)
- `tf.keras` (`fit`, `evaluate`, `predict`)
- `tf.map_fn`

Por defecto, `BatchableExtensionType` crea valores por lotes agrupando por lotes los valores `Tensor`, `CompositeTensor`, y `ExtensionType`. Si esto no es apropiado para su clase, entonces tendrá que usar `tf.experimental.ExtensionTypeBatchEncoder` para anular este comportamiento predeterminado. Por ejemplo, no será apropiado crear un lote de valores `tf.SparseTensor` simplemente apilando `values`, `indices` y campos `dense_shape` dispersos individuales; en la mayoría de los casos, no se pueden apilar estos tensores, ya que tienen formas incompatibles; e, incluso si fuera posible, el resultado no sería un `SparseTensor` válido.

**Nota**: `BatchableExtensionType` *no* definen automáticamente despachadores para `tf.stack`, `tf.concat`, `tf.slice`, etc. Si necesita que su clase sea compatible con estas API, entonces, deberá usar los decoradores de envío que se describen anteriormente.

### Ejemplo de `BatchableExtensionType`: `Network`

A modo de ejemplo, piense en una sencilla clase `Network` que se usa para equilibrar la carga, que controla cuánto trabajo queda por hacer en cada nodo y cuánto ancho de banda está disponible para mover el trabajo entre nodos:

In [ ]:
class Network(tf.experimental.ExtensionType):  # This version is not batchable.
  work: tf.Tensor       # work[n] = work left to do at node n
  bandwidth: tf.Tensor  # bandwidth[n1, n2] = bandwidth from n1->n2

net1 = Network([5., 3, 8], [[0., 2, 0], [2, 0, 3], [0, 3, 0]])
net2 = Network([3., 4, 2], [[0., 2, 2], [2, 0, 2], [2, 2, 0]])

Para hacer que este tipo sea procesable por lotes, cambie el tipo base a `BatchableExtensionType` y ajuste la forma de cada campo para que incluya dimensiones de lote opcionales. En el siguiente ejemplo, se agrega un campo `shape` para hacer un seguimiento de la forma del lote. Este campo `shape` no es necesario para `tf.data.Dataset` o `tf.map_fn`, pero *sí* para `tf.keras`.

In [ ]:
class Network(tf.experimental.BatchableExtensionType):
  shape: tf.TensorShape  # batch shape. A single network has shape=[].
  work: tf.Tensor        # work[*shape, n] = work left to do at node n
  bandwidth: tf.Tensor   # bandwidth[*shape, n1, n2] = bandwidth from n1->n2

  def __init__(self, work, bandwidth):
    self.work = tf.convert_to_tensor(work)
    self.bandwidth = tf.convert_to_tensor(bandwidth)
    work_batch_shape = self.work.shape[:-1]
    bandwidth_batch_shape = self.bandwidth.shape[:-2]
    self.shape = work_batch_shape.merge_with(bandwidth_batch_shape)

  def __repr__(self):
    return network_repr(self)

def network_repr(network):
  work = network.work
  bandwidth = network.bandwidth
  if hasattr(work, 'numpy'):
    work = ' '.join(str(work.numpy()).split())
  if hasattr(bandwidth, 'numpy'):
    bandwidth = ' '.join(str(bandwidth.numpy()).split())
  return (f"<Network shape={network.shape} work={work} bandwidth={bandwidth}>")

In [ ]:
net1 = Network([5., 3, 8], [[0., 2, 0], [2, 0, 3], [0, 3, 0]])
net2 = Network([3., 4, 2], [[0., 2, 2], [2, 0, 2], [2, 2, 0]])
batch_of_networks = Network(
    work=tf.stack([net1.work, net2.work]),
    bandwidth=tf.stack([net1.bandwidth, net2.bandwidth]))
print(f"net1={net1}")
print(f"net2={net2}")
print(f"batch={batch_of_networks}")

Luego puede usar `tf.data.Dataset` para iterar a través de un lote de redes:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(batch_of_networks)
for i, network in enumerate(dataset):
  print(f"Batch element {i}: {network}")

Y también puede usar `map_fn` para aplicar una función a cada elemento del lote:

In [ ]:
def balance_work_greedy(network):
  delta = (tf.expand_dims(network.work, -1) - tf.expand_dims(network.work, -2))
  delta /= 4
  delta = tf.maximum(tf.minimum(delta, network.bandwidth), -network.bandwidth)
  new_work = network.work + tf.reduce_sum(delta, -1)
  return Network(new_work, network.bandwidth)

tf.map_fn(balance_work_greedy, batch_of_networks)

## API TensorFlow compatibles con `ExtensionType`

### @tf.function

[`tf.function`](https://www.tensorflow.org/guide/function) es un decorador que se encarga del preprocesamiento de los gráficos de TensorFlow para funciones de Python, lo que puede mejorar considerablemente el rendimiento de su código de TensorFlow. Los valores de tipo de extensión se pueden utilizar de forma transparente con funciones decoradas con `@tf.function`.

In [ ]:
class Pastry(tf.experimental.ExtensionType):
  sweetness: tf.Tensor  # 2d embedding that encodes sweetness
  chewiness: tf.Tensor  # 2d embedding that encodes chewiness

@tf.function
def combine_pastry_features(x: Pastry):
  return (x.sweetness + x.chewiness) / 2

cookie = Pastry(sweetness=[1.2, 0.4], chewiness=[0.8, 0.2])
combine_pastry_features(cookie)

Si desea especificar de forma explícita `input_signature` para `tf.function`, puede hacerlo mediante el uso del `TypeSpec` del tipo de extensión.

In [ ]:
pastry_spec = Pastry.Spec(tf.TensorSpec([2]), tf.TensorSpec(2))

@tf.function(input_signature=[pastry_spec])
def increase_sweetness(x: Pastry, delta=1.0):
  return Pastry(x.sweetness + delta, x.chewiness)

increase_sweetness(cookie)

#### Funciones concretas

Las funciones concretas encapsulan gráficos trazados individualmente que son generados por `tf.function`. Los tipos de extensión pueden utilizarse de forma transparente con funciones concretas.


In [ ]:
cf = combine_pastry_features.get_concrete_function(pastry_spec)
cf(cookie)

### Operaciones de flujo de control

Los tipos de extensión son compatibles con las operaciones de flujo de control de TensorFlow:

- `tf.cond`
- `tf.case`
- `tf.while_loop`
- `tf.identity`


In [ ]:
# Example: using tf.cond to select between two MaskedTensors. Note that the
# two MaskedTensors don't need to have the same shape.
a = MaskedTensor([1., 2, 3], [True, False, True])
b = MaskedTensor([22., 33, 108, 55], [True, True, True, False])
condition = tf.constant(True)
print(tf.cond(condition, lambda: a, lambda: b))

In [ ]:
# Example: using tf.while_loop with MaskedTensor.
cond = lambda i, _: i < 10
def body(i, mt):
  return i + 1, mt.with_values(mt.values + 3 / 7)
print(tf.while_loop(cond, body, [0, b])[1])

### Flujo de control con Autograph

Los tipos de extensión también son compatibles con las instrucciones de flujo de control en `tf.function` (usando autograph). En el siguiente ejemplo, la instrucción `if` y las instrucciones `for` se convierten automáticamente en operaciones `tf.cond` y `tf.while_loop` que son compatibles con tipos de extensión.

In [ ]:
@tf.function
def fn(x, b):
  if b:
    x = MaskedTensor(x, tf.less(x, 0))
  else:
    x = MaskedTensor(x, tf.greater(x, 0))
  for i in tf.range(5 if b else 7):
    x = x.with_values(x.values + 1 / 2)
  return x

print(fn(tf.constant([1., -2, 3]), tf.constant(True)))
print(fn(tf.constant([1., -2, 3]), tf.constant(False)))

### Keras

[tf.keras](https://www.tensorflow.org/guide/keras) es la API de alto nivel de TensorFlow para desarrollar y entrenar modelos de aprendizaje profundo. Los tipos de extensión pueden pasarse como entradas a un modelo Keras, pasarse entre capas Keras y ser devueltos por modelos Keras. Actualmente, Keras impone dos requisitos a los tipos de extensión:

- Deben poder procesarse por lote (consulte "`ExtensionType` por lotes" más arriba).
- Deben tener un campo o propiedad con el nombre `shape`. Se asume que `shape[0]` corresponde a la dimensión del lote.

Las siguientes dos subsecciones dan ejemplos que muestran cómo se pueden usar los tipos de extensión con Keras.


#### Ejemplo de Keras: `Network`

Para el primer ejemplo, considere la clase `Network` que se definió en la sección "`ExtensionType` por lotes" anterior, que se puede usar para trabajos de equilibrio de cargas entre nodos. Su definición se repite aquí:

In [ ]:
class Network(tf.experimental.BatchableExtensionType):
  shape: tf.TensorShape  # batch shape. A single network has shape=[].
  work: tf.Tensor        # work[*shape, n] = work left to do at node n
  bandwidth: tf.Tensor   # bandwidth[*shape, n1, n2] = bandwidth from n1->n2

  def __init__(self, work, bandwidth):
    self.work = tf.convert_to_tensor(work)
    self.bandwidth = tf.convert_to_tensor(bandwidth)
    work_batch_shape = self.work.shape[:-1]
    bandwidth_batch_shape = self.bandwidth.shape[:-2]
    self.shape = work_batch_shape.merge_with(bandwidth_batch_shape)

  def __repr__(self):
    return network_repr(self)

In [ ]:
single_network = Network(  # A single network with 4 nodes.
    work=[8.0, 5, 12, 2],
    bandwidth=[[0.0, 1, 2, 2], [1, 0, 0, 2], [2, 0, 0, 1], [2, 2, 1, 0]])

batch_of_networks = Network(  # Batch of 2 networks, each w/ 2 nodes.
    work=[[8.0, 5], [3, 2]],
    bandwidth=[[[0.0, 1], [1, 0]], [[0, 2], [2, 0]]])

Puede definir una nueva capa de Keras que procese `Network`.

In [ ]:
class BalanceNetworkLayer(tf.keras.layers.Layer):
  """Layer that balances work between nodes in a network.

  Shifts work from more busy nodes to less busy nodes, constrained by bandwidth.
  """
  def call(self, inputs):
    # This function is defined above in the "Batchable `ExtensionType`s" section.
    return balance_work_greedy(inputs)

Luego, puede usar estas capas para crear un modelo simple. Para ingresar `ExtensionType` en un modelo, puede usar una capa `tf.keras.layer.Input` con un `type_spec` configurado en el `TypeSpec` del tipo de extensión. Si el modelo Keras se usará para procesar lotes, entonces, `type_spec` debe incluir la dimensión del lote.

In [ ]:
input_spec = Network.Spec(shape=None,
                          work=tf.TensorSpec(None, tf.float32),
                          bandwidth=tf.TensorSpec(None, tf.float32))
model = tf.keras.Sequential([
    tf.keras.layers.Input(type_spec=input_spec),
    BalanceNetworkLayer(),
    ])

Finalmente, puede aplicar el modelo a una única red y a un lote de redes.

In [ ]:
model(single_network)

In [ ]:
model(batch_of_networks)

#### Ejemplo de Keras: MaskedTensor

En este ejemplo, `MaskedTensor` se extiende para que sea compatible con `Keras`. `shape` se define como una propiedad que se calcula a partir del campo `values`. Keras lo obliga a agregar esta propiedad tanto al tipo de extensión como a su `TypeSpec`. `MaskedTensor` también define una variable `__name__`, que será necesaria para la serialización de `SavedModel` (a continuación).

In [ ]:
class MaskedTensor(tf.experimental.BatchableExtensionType):
  # __name__ is required for serialization in SavedModel; see below for details.
  __name__ = 'extension_type_colab.MaskedTensor'

  values: tf.Tensor
  mask: tf.Tensor

  shape = property(lambda self: self.values.shape)
  dtype = property(lambda self: self.values.dtype)

  def with_default(self, default):
    return tf.where(self.mask, self.values, default)

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  class Spec:
    def __init__(self, shape, dtype=tf.float32):
      self.values = tf.TensorSpec(shape, dtype)
      self.mask = tf.TensorSpec(shape, tf.bool)

    shape = property(lambda self: self.values.shape)
    dtype = property(lambda self: self.values.dtype)

    def with_shape(self):
      return MaskedTensor.Spec(tf.TensorSpec(shape, self.values.dtype),
                               tf.TensorSpec(shape, self.mask.dtype))

Luego, los decoradores de envío se usan para anular el comportamiento predeterminado de diversas API de TensorFlow. Dado que estas API son usadas por capas estándar de Keras (como la capa `Dense`), anularlas nos permitirá usar esas capas con `MaskedTensor`. Para los fines de este ejemplo, `matmul` para tensores enmascarados se ha definido para tratar los valores enmascarados como cero (es decir, para no incluirlos en el producto).

In [ ]:
@tf.experimental.dispatch_for_unary_elementwise_apis(MaskedTensor)
def unary_elementwise_op_handler(op, x):
 return MaskedTensor(op(x.values), x.mask)

@tf.experimental.dispatch_for_binary_elementwise_apis(
    Union[MaskedTensor, tf.Tensor],
    Union[MaskedTensor, tf.Tensor])
def binary_elementwise_op_handler(op, x, y):
  x = convert_to_masked_tensor(x)
  y = convert_to_masked_tensor(y)
  return MaskedTensor(op(x.values, y.values), x.mask & y.mask)

@tf.experimental.dispatch_for_api(tf.matmul)
def masked_matmul(a: MaskedTensor, b,
                  transpose_a=False, transpose_b=False,
                  adjoint_a=False, adjoint_b=False,
                  a_is_sparse=False, b_is_sparse=False,
                  output_type=None):
  if isinstance(a, MaskedTensor):
    a = a.with_default(0)
  if isinstance(b, MaskedTensor):
    b = b.with_default(0)
  return tf.matmul(a, b, transpose_a, transpose_b, adjoint_a,
                   adjoint_b, a_is_sparse, b_is_sparse, output_type)

Luego puede construir un modelo Keras que acepte entradas `MaskedTensor`, con las capas estándar de Keras:

In [ ]:
input_spec = MaskedTensor.Spec([None, 2], tf.float32)

masked_tensor_model = tf.keras.Sequential([
    tf.keras.layers.Input(type_spec=input_spec),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)])
masked_tensor_model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [ ]:
a = MaskedTensor([[1., 2], [3, 4], [5, 6]],
                  [[True, False], [False, True], [True, True]])
masked_tensor_model.fit(a, tf.constant([[1], [0], [1]]), epochs=3)
print(masked_tensor_model(a))

### SavedModel

Un [SavedModel](https://www.tensorflow.org/guide/saved_model) es un programa de TensorFlow serializado, que incluye tanto los pesos como la computación. Se puede desarrollar a partir de un modelo Keras o de un modelo personalizado. En cualquier caso, los tipos de extensión se pueden usar de forma transparente con las funciones y los métodos definidos por un SavedModel.

SavedModel puede guardar modelos, capas y funciones que procesen tipos de extensión, siempre y cuando los tipos de extensión tengan un campo `__name__`. Este nombre se usa para registrar el tipo de extensión, para que se pueda localizar cuando se cargue el modelo.

#### Ejemplo: cómo guardar un modelo Keras

Los modelos Keras que usan tipos de extensión se pueden guardar mediante el uso de `SavedModel`.

In [ ]:
masked_tensor_model_path = tempfile.mkdtemp()
tf.saved_model.save(masked_tensor_model, masked_tensor_model_path)
imported_model = tf.saved_model.load(masked_tensor_model_path)
imported_model(a)

#### Ejemplo: cómo guardar un modelo personalizado

SavedModel también se puede usar para guardar subclases `tf.Module` personalizadas con funciones que procesen tipos de extensión.

In [ ]:
class CustomModule(tf.Module):
  def __init__(self, variable_value):
    super().__init__()
    self.v = tf.Variable(variable_value)

  @tf.function
  def grow(self, x: MaskedTensor):
    """Increase values in `x` by multiplying them by `self.v`."""
    return MaskedTensor(x.values * self.v, x.mask)

module = CustomModule(100.0)

module.grow.get_concrete_function(MaskedTensor.Spec(shape=None,
                                                    dtype=tf.float32))
custom_module_path = tempfile.mkdtemp()
tf.saved_model.save(module, custom_module_path)
imported_model = tf.saved_model.load(custom_module_path)
imported_model.grow(MaskedTensor([1., 2, 3], [False, True, False]))

#### Cómo cargar un SavedModel cuando el `ExtensionType` no está disponible

Si carga un `SavedModel` que usa un `ExtensionType`, pero ese `ExtensionType` no está disponible (es decir, no ha sido importado), entonces recibirá una advertencia y TensorFlow volverá a utilizar un objeto "tipo de extensión anónimo". Este objeto tendrá los mismos campos que el tipo original, pero no tendrá ninguna otra personalización que haya agregado al tipo, como métodos personalizados o propiedades.

#### Cómo usar `ExtensionType` con TensorFlow Serving

Actualmente, [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) (y otros consumidores del diccionario de "firmas" de SavedModel) requieren que todas las entradas y salidas sean tensores sin procesar. Si desea usar TensorFlow Serving con un modelo que use tipos de extensión, puede agregar métodos de envoltura que compongan o descompongan valores de tipos de extensión a partir de tensores. Por ejemplo:

In [ ]:
class CustomModuleWrapper(tf.Module):
  def __init__(self, variable_value):
    super().__init__()
    self.v = tf.Variable(variable_value)

  @tf.function
  def var_weighted_mean(self, x: MaskedTensor):
    """Mean value of unmasked values in x, weighted by self.v."""
    x = MaskedTensor(x.values * self.v, x.mask)
    return (tf.reduce_sum(x.with_default(0)) /
            tf.reduce_sum(tf.cast(x.mask, x.dtype)))

  @tf.function()
  def var_weighted_mean_wrapper(self, x_values, x_mask):
    """Raw tensor wrapper for var_weighted_mean."""
    return self.var_weighted_mean(MaskedTensor(x_values, x_mask))

module = CustomModuleWrapper([3., 2., 8., 5.])

module.var_weighted_mean_wrapper.get_concrete_function(
    tf.TensorSpec(None, tf.float32), tf.TensorSpec(None, tf.bool))
custom_module_path = tempfile.mkdtemp()
tf.saved_model.save(module, custom_module_path)
imported_model = tf.saved_model.load(custom_module_path)
x = MaskedTensor([1., 2., 3., 4.], [False, True, False, True])
imported_model.var_weighted_mean_wrapper(x.values, x.mask)

### `Dataset`

[`tf.data`](https://www.tensorflow.org/guide/data) es una API que le permite desarrollar canalizaciones de entradas complejas a partir de piezas reutilizables simples. Su estructura de datos principal es `tf.data.Dataset`, que representa una secuencia de elementos, en la que cada elemento consiste de uno o más componentes.

#### Cómo desarrollar `Dataset` con tipos de extensión

Los conjuntos de datos se pueden desarrollar a partir de valores de tipo de extensión si se usa `Dataset.from_tensors`, `Dataset.from_tensor_slices` o `Dataset.from_generator`:

In [ ]:
ds = tf.data.Dataset.from_tensors(Pastry(5, 5))
iter(ds).next()

In [ ]:
mt = MaskedTensor(tf.reshape(range(20), [5, 4]), tf.ones([5, 4]))
ds = tf.data.Dataset.from_tensor_slices(mt)
for value in ds:
  print(value)

In [ ]:
def value_gen():
  for i in range(2, 7):
    yield MaskedTensor(range(10), [j%i != 0 for j in range(10)])

ds = tf.data.Dataset.from_generator(
    value_gen, output_signature=MaskedTensor.Spec(shape=[10], dtype=tf.int32))
for value in ds:
  print(value)

#### Agrupar o desagrupar `Dataset` por lotes con tipos de extensión

Los conjuntos de datos con tipos de extensión se pueden agrupar o desagrupar por lotes si se usa `Dataset.batch` y `Dataset.unbatch`.

In [ ]:
batched_ds = ds.batch(2)
for value in batched_ds:
  print(value)

In [ ]:
unbatched_ds = batched_ds.unbatch()
for value in unbatched_ds:
  print(value)